 # Variables

In [1]:
# For Train & Eval
arg = "origin"
dataset = "BEST"
task = "origami"
lap = "1"
split = "1"
cuda = [0]

# For Eval
weight_epoch = "0"
is_best = True

# Import 

In [2]:
import argparse
import yaml
import torch
import time
import os
import glob

from addict import Dict

from main.util import make_dirs, AverageMeter, data_augmentation, accuracy, sec2str
from main.loss import diversity_loss, disparity_loss
from main.dataset import SkillDataSet
from main.model import RAAN
from main.trainrun import Train_Runner, earlystopping

# Args

In [3]:
# yaml args
args = Dict(yaml.safe_load(open(os.path.join('args',arg+'.yaml'))))

# show args 
print(('\n''[Arguements]\n''{0}\n''\n'.format(args)))

# device setting
cuda = list(map(str, cuda))
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(cuda)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


[Arguements]
{'data_path': '../../local/dataset/skill', 'ckpt_path': './ckpt/models', 'writer_path': './ckpt/logs', 'num_samples': 400, 'input_size': 1024, 'output_size': 256, 'attention': True, 'num_filters': 3, 'diversity_loss': True, 'disparity_loss': 'v1', 'rank_aware_loss': 'v1', 'lambda_param': 0.1, 'm1': 1.0, 'm2': 0.05, 'm3': 0.15, 'temporal_model': False, 'num_layers': 9, 'num_f_maps': 256, 'epochs': 2000, 'transform': True, 'batch_size': 128, 'lr': 0.0001, 'workers': 4, 'start_epoch': 1, 'print_freq': 5, 'eval_freq': 10, 'ckpt_freq': 5, 'earlystopping': 20}




# Paths

In [4]:
# path
if dataset == "BEST":
    train_txt = "train.txt"
    val_txt = "test.txt"
    savedir = os.path.join(dataset, task, arg, "lap_"+lap)
elif dataset == "EPIC-Skills":
    train_txt = "train_split" + split + ".txt"
    val_txt = "test_split" + split + ".txt"
    savedir = os.path.join(dataset, task, arg, split, "lap_"+lap)
    
train_list = os.path.join(args.data_path, dataset, 'splits', task, train_txt)
valid_list = os.path.join(args.data_path, dataset, 'splits', task, val_txt)
feature_path = os.path.join(args.data_path, dataset, 'features', task)
ckptdir = os.path.join(args.ckpt_path, savedir)

# paths dict
paths = {'train_list': train_list, 'valid_list': valid_list,
         'feature_path': feature_path, "ckptdir" : ckptdir}


# Models

In [5]:
### attention branch ###
# → 2branch(pos and neg)
if args.disparity_loss and args.rank_aware_loss:
    model_attention = {'pos': None, 'neg': None}
# → 1branch
else:
    model_attention = {'att': None}
# attention model
for k in model_attention.keys():
    model_attention[k] = RAAN(args, uniform=False, tcn=args.temporal_model)
    model_attention[k] = model_attention[k].to(device)

### uniform branch ###
if args.disparity_loss:
    model_uniform = RAAN(args, uniform=True, tcn=False)
    model_uniform = model_uniform.to(device)
else:
    model_uniform = None

# models dict
models = {"attention" : model_attention , "uniform" : model_uniform}
att_branches = models["attention"].keys()

# Dataloader

In [6]:
# train_data = train_vid_list.txt 
train_loader = torch.utils.data.DataLoader(
    SkillDataSet(paths["feature_path"], paths["train_list"], ftr_tmpl='{}_{}.npz'),
    batch_size=args.batch_size,
    shuffle=True,
    num_workers=args.workers,
    pin_memory=True)

# validation_data = test_vid_list.txt
valid_loader = torch.utils.data.DataLoader(
    SkillDataSet(paths["feature_path"], paths["valid_list"], ftr_tmpl='{}_{}.npz'),
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.workers,
    pin_memory=True)

# dataloaders dict
dataloaders = {"train" : train_loader, "valid" : valid_loader}

#iterator
train_iterator = iter(dataloaders["train"])
valid_iterator = iter(dataloaders["valid"])

# Loss

In [7]:
# lossses
ranking_loss = torch.nn.MarginRankingLoss(margin=args.m1)

# criterions dict
criterions = {"ranking" : ranking_loss, "disparity" : disparity_loss, "diversity" : diversity_loss}

# ========
# 1. Train  
# ========

# 1.1 Without uniform

In [80]:
# model train mode
model = models["attention"][list(att_branches)[0]]
model.train()

RAAN(
  (att_net): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=1024, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=1, bias=True)
      (3): Softmax(dim=1)
    )
    (1): Sequential(
      (0): Linear(in_features=1024, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=1, bias=True)
      (3): Softmax(dim=1)
    )
    (2): Sequential(
      (0): Linear(in_features=1024, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=1, bias=True)
      (3): Softmax(dim=1)
    )
  )
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=1, bias=True)
    (1): Tanh()
  )
)

In [81]:
# iterator
vid_pos, vid_neg, vid_list = next(train_iterator)
batch_size = vid_pos.size(0)
vid_pos_gpu = vid_pos.to(device)
vid_neg_gpu = vid_neg.to(device)

# make target label
target = torch.ones(vid_pos.size(0))
target = target.to(device)

# calc score, attention
score_pos, att_pos = model(vid_pos_gpu)
score_neg, att_neg = model(vid_neg_gpu)

# mean all filter
score_pos = score_pos.mean(dim=1)
score_neg = score_neg.mean(dim=1)

In [89]:
vid_pos.size(0)

8

# 1.2 With uniform

In [8]:
# model train mode
for k in att_branches:
    models["attention"][k].train()
models["uniform"].train()

RAAN(
  (att_net): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=1024, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=1, bias=True)
      (3): Softmax(dim=1)
    )
  )
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=1, bias=True)
    (1): Tanh()
  )
)

In [19]:
# iterator
vid_pos, vid_neg, vid_list = next(train_iterator)
batch_size = vid_pos.size(0)
vid_pos_gpu = vid_pos.to(device)
vid_neg_gpu = vid_neg.to(device)

# make target label
target = torch.ones(vid_pos.size(0))
target = target.to(device)

# 1.3 Valid

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ヒートマップ表示
plt.figure(figsize=(20,2))
plt.imshow(a1,interpolation='nearest',vmin=0,vmax=1/200,cmap='jet')
plt.colorbar()
plt.show()

# ===========
# 2. Evaluate  
# ===========

In [ ]:
# attach model weight
if is_best:
    weight_path = glob.glob(os.path.join(ckptdir, 'best_score*'))[0] 
else::
    weight_path = glob.glob(os.path.join(ckptdir, 'epoch_' + str(epoch).zfill(4) + '*'))[0]
    
for k in att_branches:
    models["attention"][k].load_state_dict(torch.load(weight_path)["state_dict_" + k])
        
print(torch.load(weight_path)["prec_score"])
print(torch.load(weight_path)["epoch"])

In [9]:
# iterator
vid_pos, vid_neg, vid_list = next(valid_iterator)
batch_size = vid_pos.size(0)
vid_pos_gpu = vid_pos.to(device)
vid_neg_gpu = vid_neg.to(device)
score_list["pos"].extend(vid_list["pos"]) 
score_list["neg"].extend(vid_list["neg"])

# calc score, attention
all_score_pos, all_score_neg, score_pos, score_neg, att_pos, att_neg = {}, {}, {}, {}, {}, {}
final_score_pos = torch.zeros(batch_size).to(device)
final_score_neg = torch.zeros(batch_size).to(device)
for k in att_branches:
    all_score_pos[k], att_pos[k] = models["attention"][k](vid_pos_gpu)
    all_score_neg[k], att_neg[k] = models["attention"][k](vid_neg_gpu)
    score_pos[k] = all_score_pos[k].mean(dim=1)
    score_neg[k] = all_score_neg[k].mean(dim=1)
    att_pos[k] = att_pos[k].mean(dim=2)
    att_neg[k] = att_neg[k].mean(dim=2)
    if att_list["pos_"+k] is None:
        att_list["pos_"+k] = np.squeeze(att_pos[k].cpu().detach().numpy(), 2)
        att_list["neg_"+k] = np.squeeze(att_neg[k].cpu().detach().numpy(), 2)
    else:
        att_list["pos_"+k] = np.concatenate((att_list["pos_"+k], np.squeeze(att_pos[k].cpu().detach().numpy(), 2)), axis=0)
        att_list["neg_"+k] = np.concatenate((att_list["neg_"+k], np.squeeze(att_neg[k].cpu().detach().numpy(), 2)), axis=0)
    final_score_pos += score_pos[k].data
    final_score_neg += score_neg[k].data
score_list["pos_score"].extend(final_score_pos.cpu().numpy())
score_list["neg_score"].extend(final_score_neg.cpu().numpy())

In [12]:
correct_list = [pos_score_list[i]>neg_score_list[i] for i in range(len(pos_score_list))]

torch.Size([84, 400, 3, 1])

In [ ]:
# dataframe
eval_df = pd.DataFrame({'vid_pos' : score_list["pos"], 
                        'vid_pos_score' : score_list["pos_score"], 
                        'vid_neg' : score_list["neg"], 
                        'vid_neg_score' : score_list["neg_score"], 
                        'correct' : correct_list
                       })
eval_df.index = np.arange(1,len(pos_score_list)+1)
eval_df.to_csv("test.csv")

In [ ]:
for k in att_list:
    att_df = pd.DataFrame(att_list[k],
                          index = score_list[k[0:3]]
    )
    att_df.columns = np.arange(1,401)
    att_df.to_csv(os.path.join("a.csv"))